<a href="https://colab.research.google.com/github/Dutra-Apex/llm-joc/blob/main/video-extraction/transcript_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Description:  
This program will summarize a youtube video using the Mistral LLM.  The summaries are based on the youtube auto-genarated closed captioning transcript stored with the video.  This program divides the transcript into time length sections and summarizes each section.  It then summarizes the entire video based on summarizing all the summaries together.

In [33]:
# Format the output when printing in colab
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
      white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)


In [34]:
import locale

def getpreferredencoding(do_setlocale = True):
  return "UTF-8"

locale.getpreferredencoding = getpreferredencoding
locale.getdefaultlocale()


('en_US', 'UTF-8')

# **Create LLM**

The following code is from https://blog.gopenai.com/bye-bye-llama-2-mistral-7b-is-taking-over-get-started-with-mistral-7b-instruct-1504ff5f373c


## Step 1.  Import Libraries

LLM and LangChain libraries

In [3]:
# -q quiets the output
!pip install -qU kaleido python-multipart uvicorn fastapi==0.99.1 typing-extensions==4.5 torch==2.1
!pip install -qU accelerate bitsandbytes langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.29 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.16.3 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.1.0 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.1.0 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.1.0 which is 

Import Libraries

In [4]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import tensorflow
import torch
import pandas as pd
import math
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


device = 'cuda' if torch.cuda.is_available() else 'cpu'

If want to load model from google drive

In [5]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
model_path = "/content/drive/MyDrive/Mistral-7B-Instruct-v0.2"
# model_path = "mistralai/Mistral-7B-Instruct-v0.2"

## Step 2.  Download the Mistral 7B Instruct Model and Tokenizer

In [7]:
model = AutoModelForCausalLM.from_pretrained(model_path, load_in_4bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_path)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
#Create pipeline for text generation
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=2000,
    do_sample = True
)

In [9]:
#Create insance of llm
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


#### Get list of video ids from playlist

In [35]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.7 MB/s eta 0:00:00


In [36]:
from pytube import Playlist

In [37]:
# Create a Playlist object
playlist_url = "https://www.youtube.com/playlist?list=PLuePfAWKCLvXDmUCkglj2na2f4TCvCfLg"
playlist = Playlist(playlist_url)

In [38]:
# Get video titles
video_data = []
for video in playlist.videos:
    video_data.append([video.video_id, video.title])

video_titles_df = pd.DataFrame(video_data, columns=['video_id', 'title'])

In [39]:
print(video_titles_df.head())

      video_id                                              title
0  QwguYMC9doI        2024-03-28T00:59:31Z - Web Wednesdays Q & A
1  o7vB2gaM2YE  2024-03-27T23:30:10Z - Data Science / Open Q &...
2  RHH9Rk8_9B0  2024-03-27T21:57:07Z - Web Wed for Explorers &...
3  mglK874_zlM  2024-03-27T16:58:23Z - Web Wed for Explorers &...
4  s6JUoFCKcHs  2024-03-27T01:24:49Z - Python Party & Dev-in-T...


##Get Video Transcripts


In [13]:
!pip install youtube-transcript-api

In [14]:
# importing the module
from youtube_transcript_api import YouTubeTranscriptApi

In [15]:
# getting the transcript for a video

video_id = "NoXCHb9ydxQ"  #From 2024-03-20T01:00:51Z - Python Party & Dev-in-Training Updates. https://www.youtube.com/watch?v=NoXCHb9ydxQ
srt = YouTubeTranscriptApi.get_transcript(video_id)

In [17]:
#Convert youtube list of objects to pandas dataframe
video_df = pd.DataFrame(srt, columns=["start", "text"])

In [18]:
print(video_df.head())

   start                                     text
0  16.00  hello hello hello can everybody hear me
1  18.16      okay welcome welcome I hear you yes
2  21.96  awesome Joseph awesome good to hear I'm
3  24.80     gonna turn my volume up a little bit
4  26.92      there we go excellent some familiar


In [11]:
# Function that creates a list of dataframes.  Each dataframe in the list contains a block of video trancription based on the time given by the section_minutes parameter.
def get_section_frames(df, section_minutes = 10, back_time_secs = 0):
  section_length = section_minutes * 60
  end_time = df["start"].max()
  n_sections = math.ceil(end_time / section_length)
  section_list = []
  for i in range(n_sections):
    section_df = df[(df["start"]> i* section_length - back_time_secs) & (df["start"] < (i+1)*section_length)]
    section_list.append(section_df)
  return section_list

In [12]:
# Function that creates a list of text sections from a list of dataframes.  Each text section is made by joining all the text within a dataframe text column.
def get_text_from_frames(df_list):
  sections_text = []
  for df in df_list:
    section_text = ''
    for index, row in df.iterrows():
      section_text += row["text"] + " "
    sections_text.append(section_text)
  return sections_text

In [28]:
# Convert dataframe to a list of section text
section_minutes = 10
sect_list = get_text_from_frames(get_section_frames(video_df, section_minutes = section_minutes))

## Get Summaries

In [10]:
#Prompt used to summarize sections within a video
def get_prompt():
  return f"""[INSTRUCT] Summarize the following video transcript into a concise paragraph.

{context}

[/INSTRUCT]"""

In [22]:
summaries = []
for context in sect_list:
  prompt = get_prompt()
  summary = llm(prompt)
  summaries.append(summary)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id`

In [24]:
# Combine all the summaries to have the model summarize the summaries
combined_summary = "\n".join(summaries)

In [25]:
summary_prompt =  f"""[INSTRUCT] Summarize the following compiled summaries in a concise paragraph.

{combined_summary}

[/INSTRUCT]"""

In [26]:
complete_summary = llm(summary_prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [29]:
#Print summaries
print("Summary of the video:")
print(complete_summary)
print()
print("Section summaries:")
for i in range(len(summaries)):
  print(f"{i*section_minutes} - {(i+1)*section_minutes} minutes.", summaries[i])
  print()

Summary of the video:
 During the coding training session, the speaker welcomed attendees and shared their excitement about the topic of the night: dictionaries. They encouraged participants to share their achievements in their coding journey and emphasized the importance of persistence and resilience. The session then transitioned into a training on dictionaries, a data structure used to store key-value pairs. Dictionaries offer more flexibility than lists and are commonly used in web development. The speaker demonstrated creating a dictionary in Python and discussed its advantages, such as faster lookups and the ability to create variables on the fly. They also compared dictionaries to lists and emphasized their importance in programming. Participants shared their experiences with coding challenges, discussing issues they encountered when running code locally versus in production environments. They advised taking a step-by-step approach to problem-solving and emphasized the importanc

In [31]:
# Save summaries to file
filename = "video_summary.txt"
with open(filename, "w") as file:
  file.write("Summary of the video:\n")
  file.write(complete_summary)
  file.write("\nSection summaries:\n")
  for i in range(len(summaries)):
    file.write(f"{i*section_minutes} - {(i+1)*section_minutes} minutes. {summaries[i]} \n")